In [2]:
from tensorflow.keras.models import load_model
model = load_model('sign_language_main_1.h5')


In [2]:
"""import cv2
import numpy as np
from tensorflow.keras.models import load_model
import time

# Load the pre-trained model
model = load_model('sign_language_main_1.h5')

# Map class indices to letters
class_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 
                 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 
                 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'del', 27: 'nothing', 28: 'space'}

# Open the camera
cap = cv2.VideoCapture(0)

# Initialize variables for storing and displaying letters and words
detected_letters = []
detected_word = ""
start_time = None

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    # Increase the camera size
    frame = cv2.resize(frame, (1000, 600))

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply hand detection logic here (use your hand detection code)
    # For demonstration purposes, let's assume hand_roi is the region of interest containing the hand
    # Calculate the center of the frame
    frame_center = (frame.shape[1] // 2, frame.shape[0] // 2)
    rect_size = 250  # Size of the hand detection rectangle
    top_left = (frame_center[0] - rect_size // 2, frame_center[1] - rect_size // 2)
    bottom_right = (top_left[0] + rect_size, top_left[1] + rect_size)

    # Resize and preprocess the hand ROI
    hand_roi = frame[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
    processed_hand = cv2.resize(hand_roi, (224, 224))
    processed_hand = np.expand_dims(processed_hand, axis=0)
    processed_hand = processed_hand / 255.0  # Normalize the pixel values

    # Get the prediction from the model
    prediction = model.predict(processed_hand)
    predicted_class = np.argmax(prediction)

    # Get the predicted sign based on the class mapping
    predicted_sign = class_mapping.get(predicted_class, 'Unknown')

    # Draw a rectangle around the hand
    cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)  # Hand detection rectangle at the center

    # Display the predicted sign with background color
    sign_bg_color = (255, 0, 0)  # Specify the background color
    sign_text_color = (255, 255, 255)  # Specify the text color
    cv2.rectangle(frame, (10, 10), (200, 50), sign_bg_color, -1)
    cv2.putText(frame, predicted_sign, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, sign_text_color, 2, cv2.LINE_AA)

    # Check if the detected sign is 'nothing' (hand not detected)
    if predicted_sign == 'nothing':
        # Reset the timer when hand is not detected
        start_time = None
    else:
        # If hand is detected, start or update the timer
        if start_time is None:
            start_time = time.time()
        else:
            # Check if the detected sign is displayed for more than 3 seconds
            elapsed_time = time.time() - start_time
            if elapsed_time >= 3:
                if predicted_sign == 'space':
                    detected_letters.append('_')  # Add underscore when SPACE is detected
                elif predicted_sign == 'del':
                    if detected_letters:
                        detected_letters.pop()  # Remove the last letter from the list
                    detected_word = ''.join(detected_letters)  # Update the detected word
                else:
                    detected_letters.append(predicted_sign)
                detected_word = ''.join(detected_letters)
                start_time = None  # Reset the timer

    # Draw a rectangle around the detected word
    word_bg_color = (0, 0, 255)  # Specify the background color
    word_text_color = (255, 255, 255)  # Specify the text color
    cv2.rectangle(frame, (10, 60), (500, 90), word_bg_color, -1)
    cv2.putText(frame, f'Detected Word: {detected_word}', (20, 80 + 5), cv2.FONT_HERSHEY_SIMPLEX, 1, word_text_color, 2, cv2.LINE_AA)

    # Display the frame with the hand detection and prediction
    cv2.imshow('Hand Detection and Prediction', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
"""

1/1 [==============================] - 0s 28ms/step


In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import time
import pyttsx3  # Import the pyttsx3 library for text-to-speech

# Load the pre-trained model
model = load_model('sign_language_main_1.h5')

# Map class indices to letters and gestures (including the open palm gesture, assumed to be 'open_palm')
class_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 
                 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 
                 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'del', 27: 'nothing', 28: 'space', 29: 'open_palm'}

# Initialize the text-to-speech engine
tts_engine = pyttsx3.init()

# Set TTS properties (optional)
tts_engine.setProperty('rate', 150)  # Speed of speech
tts_engine.setProperty('volume', 1.0)  # Volume level (0.0 to 1.0)

# Test TTS engine to ensure it's working
tts_engine.say("System initialized successfully.")
tts_engine.runAndWait()

# Open the camera
cap = cv2.VideoCapture(0)

# Initialize variables for storing letters, words, and sentences
detected_letters = []
detected_word = ""
start_time = None
previous_sign = ""  # To avoid repeating the same letter
previous_word = ""  # To avoid pronouncing the same word twice

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    # Increase the camera size
    frame = cv2.resize(frame, (1000, 600))

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Hand detection logic and frame setup
    frame_center = (frame.shape[1] // 2, frame.shape[0] // 2)
    rect_size = 250  # Size of the hand detection rectangle
    top_left = (frame_center[0] - rect_size // 2, frame_center[1] - rect_size // 2)
    bottom_right = (top_left[0] + rect_size, top_left[1] + rect_size)

    # Resize and preprocess the hand ROI
    hand_roi = frame[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
    processed_hand = cv2.resize(hand_roi, (224, 224))
    processed_hand = np.expand_dims(processed_hand, axis=0)
    processed_hand = processed_hand / 255.0  # Normalize the pixel values

    # Get the prediction from the model
    prediction = model.predict(processed_hand)
    predicted_class = np.argmax(prediction)

    # Get the predicted sign based on the class mapping
    predicted_sign = class_mapping.get(predicted_class, 'Unknown')

    # Debug: Print the detected sign
    print(f"Detected sign: {predicted_sign}")

    # Draw a rectangle around the hand
    cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)  # Hand detection rectangle at the center

    # Display the predicted sign with background color
    sign_bg_color = (255, 0, 0)  # Specify the background color
    sign_text_color = (255, 255, 255)  # Specify the text color
    cv2.rectangle(frame, (10, 10), (200, 50), sign_bg_color, -1)
    cv2.putText(frame, predicted_sign, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, sign_text_color, 2, cv2.LINE_AA)

    # Check if the detected sign is 'nothing' (hand not detected)
    if predicted_sign == 'nothing':
        start_time = None  # Reset the timer when hand is not detected
    else:
        # If hand is detected, start or update the timer
        if start_time is None:
            start_time = time.time()
        else:
            # Check if the detected sign is displayed for more than 3 seconds
            elapsed_time = time.time() - start_time
            if elapsed_time >= 3:
                if predicted_sign == 'space':  # Pronounce the word when 'space' is detected
                    detected_word = ''.join(detected_letters).replace('_', '').strip()
                    print(f"Detected word: {detected_word}")  # Debug: Print the formed word
                    if detected_word and detected_word != previous_word:
                        print(f"Pronouncing: {detected_word}")  # Debug: Print what will be pronounced
                        tts_engine.say(detected_word)  # Pronounce the entire word
                        tts_engine.runAndWait()  # Wait for the speech to complete
                        previous_word = detected_word  # Store the pronounced word
                    detected_letters = []  # Clear letters after pronouncing the word
                elif predicted_sign == 'del':  # Handle delete
                    if detected_letters:
                        detected_letters.pop()  # Remove the last letter from the list
                elif predicted_sign == 'open_palm':  # Optionally handle open palm separately
                    pass  # Do something for open palm gesture if needed
                else:
                    # Ensure the same sign is not processed multiple times within the time frame
                    if predicted_sign != previous_sign:
                        detected_letters.append(predicted_sign)  # Collect the letters
                        previous_sign = predicted_sign  # Avoid repeating the same letter

                start_time = None  # Reset the timer after processing the sign

    # Draw a rectangle around the detected word
    word_bg_color = (0, 0, 255)  # Specify the background color
    word_text_color = (255, 255, 255)  # Specify the text color
    cv2.rectangle(frame, (10, 60), (700, 90), word_bg_color, -1)
    cv2.putText(frame, f'Detected Word: {"".join(detected_letters)}', (20, 80 + 5), cv2.FONT_HERSHEY_SIMPLEX, 1, word_text_color, 2, cv2.LINE_AA)

    # Display the frame with the hand detection and prediction
    cv2.imshow('Hand Detection and Prediction', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 346ms/step
Detected sign: nothing
1/1 [==============================] - 0s 56ms/step
Detected sign: nothing
1/1 [==============================] - 0s 43ms/step
Detected sign: nothing
1/1 [==============================] - 0s 44ms/step
Detected sign: nothing
1/1 [==============================] - 0s 49ms/step
Detected sign: nothing
1/1 [==============================] - 0s 41ms/step
Detected sign: nothing
1/1 [==============================] - 0s 34ms/step
Detected sign: nothing
1/1 [==============================] - 0s 46ms/step
Detected sign: nothing
1/1 [==============================] - 0s 41ms/step
Detected sign: nothing
1/1 [==============================] - 0s 40ms/step
Detected sign: nothing
1/1 [==============================] - 0s 57ms/step
Detected sign: nothing
1/1 [==============================] - 0s 35ms/step
Detected sign: nothing
1/1 [==============================] - 0s 36ms/step
Detected sign: nothing
1/1 [===================

In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import time
import pyttsx3  # Import the pyttsx3 library for text-to-speech

# Load the pre-trained model
model = load_model('sign_language_main_1.h5')

# Map class indices to letters and gestures (including the open palm gesture)
class_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 
                 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 
                 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'del', 
                 27: 'nothing', 28: 'space', 29: 'open_palm'}

# Initialize the text-to-speech engine
tts_engine = pyttsx3.init()
tts_engine.setProperty('rate', 150)  # Speed of speech
tts_engine.setProperty('volume', 1.0)  # Volume level (0.0 to 1.0)

# Test TTS engine to ensure it's working
tts_engine.say("System initialized successfully.")
tts_engine.runAndWait()

# Open the camera
cap = cv2.VideoCapture(0)

# Initialize variables for storing letters, words, and sentences
detected_letters = []
detected_word = ""
start_time = None
previous_sign = ""  # To avoid repeating the same letter
previous_word = ""  # To avoid pronouncing the same word twice
final_detected_text = ""  # Store final text for printing after camera closes

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    # Increase the camera size
    frame = cv2.resize(frame, (1000, 600))

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Hand detection logic and frame setup
    frame_center = (frame.shape[1] // 2, frame.shape[0] // 2)
    rect_size = 250  # Size of the hand detection rectangle
    top_left = (frame_center[0] - rect_size // 2, frame_center[1] - rect_size // 2)
    bottom_right = (top_left[0] + rect_size, top_left[1] + rect_size)

    # Resize and preprocess the hand ROI
    hand_roi = frame[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
    processed_hand = cv2.resize(hand_roi, (224, 224))
    processed_hand = np.expand_dims(processed_hand, axis=0)
    processed_hand = processed_hand / 255.0  # Normalize the pixel values

    # Get the prediction from the model
    prediction = model.predict(processed_hand)
    predicted_class = np.argmax(prediction)
    predicted_sign = class_mapping.get(predicted_class, 'Unknown')

    # Debug: Print the detected sign
    print(f"Detected sign: {predicted_sign}")

    # Draw a rectangle around the hand
    cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)

    # Display the predicted sign with background color
    sign_bg_color = (255, 0, 0)  # Background color
    sign_text_color = (255, 255, 255)  # Text color
    cv2.rectangle(frame, (10, 10), (200, 50), sign_bg_color, -1)
    cv2.putText(frame, predicted_sign, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, sign_text_color, 2, cv2.LINE_AA)

    # Check if the detected sign is 'nothing' (hand not detected)
    if predicted_sign == 'nothing':
        start_time = None  # Reset the timer when hand is not detected
    else:
        # If hand is detected, start or update the timer
        if start_time is None:
            start_time = time.time()
        else:
            # Check if the detected sign is displayed for more than 3 seconds
            elapsed_time = time.time() - start_time
            if elapsed_time >= 3:
                if predicted_sign == 'space':  # Pronounce the word when 'space' is detected
                    detected_word = ''.join(detected_letters).replace('_', '').strip()
                    print(f"Detected word: {detected_word}")
                    if detected_word and detected_word != previous_word:
                        print(f"Pronouncing: {detected_word}")
                        tts_engine.say(detected_word)
                        tts_engine.runAndWait()
                        previous_word = detected_word
                        final_detected_text += detected_word + " "  # Append word to final detected text
                    detected_letters = []
                elif predicted_sign == 'del':  # Handle delete
                    if detected_letters:
                        detected_letters.pop()
                elif predicted_sign == 'open_palm':  # Optionally handle open palm separately
                    pass
                else:
                    # Ensure the same sign is not processed multiple times within the time frame
                    if predicted_sign != previous_sign:
                        detected_letters.append(predicted_sign)
                        previous_sign = predicted_sign

                start_time = None  # Reset the timer after processing the sign

    # Draw a rectangle around the detected word
    word_bg_color = (0, 0, 255)  # Background color
    word_text_color = (255, 255, 255)  # Text color
    cv2.rectangle(frame, (10, 60), (700, 90), word_bg_color, -1)
    cv2.putText(frame, f'Detected Word: {"".join(detected_letters)}', (20, 80 + 5), cv2.FONT_HERSHEY_SIMPLEX, 1, word_text_color, 2, cv2.LINE_AA)

    # Display the frame with the hand detection and prediction
    cv2.imshow('Hand Detection and Prediction', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

# Print the final detected text after closing the camera
print("Final Detected Text:", final_detected_text)


1/1 [==============================] - 0s 270ms/step
Detected sign: nothing
1/1 [==============================] - 0s 41ms/step
Detected sign: nothing
1/1 [==============================] - 0s 45ms/step
Detected sign: nothing
1/1 [==============================] - 0s 47ms/step
Detected sign: nothing
1/1 [==============================] - 0s 46ms/step
Detected sign: nothing
1/1 [==============================] - 0s 41ms/step
Detected sign: nothing
1/1 [==============================] - 0s 36ms/step
Detected sign: nothing
1/1 [==============================] - 0s 43ms/step
Detected sign: nothing
1/1 [==============================] - 0s 39ms/step
Detected sign: nothing
1/1 [==============================] - 0s 40ms/step
Detected sign: nothing
1/1 [==============================] - 0s 38ms/step
Detected sign: nothing
1/1 [==============================] - 0s 39ms/step
Detected sign: nothing
1/1 [==============================] - 0s 42ms/step
Detected sign: nothing
1/1 [===================

In [5]:
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import cv2
import os

# Load the pre-trained model
model = load_model('sign_language_main_1.h5')

# Define the class mapping dictionary
class_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 
                 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 
                 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'del', 
                 27: 'nothing', 28: 'space', 29: 'open_palm'}

# Reverse the class mapping for easy lookup
reverse_mapping = {v: k for k, v in class_mapping.items()}

# Function to evaluate the model
def evaluate_model(test_dir):
    true_labels = []
    predictions = []

    # Iterate over all files in the test directory
    for img_name in os.listdir(test_dir):
        # Only process image files
        if img_name.endswith('.jpg') or img_name.endswith('.png'):
            img_path = os.path.join(test_dir, img_name)

            # Extract the true label from the filename (assuming format "A_test.jpg")
            true_label_letter = img_name[0].upper()
            true_label = reverse_mapping.get(true_label_letter, None)
            
            # Skip if label is not found in the mapping
            if true_label is None:
                continue

            # Load and preprocess the image
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))  # Resize to match model input size
            img = np.expand_dims(img, axis=0) / 255.0  # Normalize and add batch dimension
            
            # Make a prediction
            pred = model.predict(img)
            pred_class = np.argmax(pred)

            # Append true label and predicted label
            true_labels.append(true_label)
            predictions.append(pred_class)

    # Calculate accuracy and display a detailed report
    accuracy = accuracy_score(true_labels, predictions)
    print("Model Accuracy:", accuracy)
    
    # Filter target names to only include classes present in the predictions
    target_names = [class_mapping[i] for i in set(true_labels + predictions)]
    print("Detailed Classification Report:\n", classification_report(true_labels, predictions, target_names=target_names))

# Run the evaluation with the specified directory
evaluate_model('extracted_archive_11/asl_alphabet_test/asl_alphabet_test')


1/1 [==============================] - 0s 97ms/step
Model Accuracy: 0.8571428571428571
Detailed Classification Report:
               precision    recall  f1-score   support

           A       1.00      1.00      1.00         1
           B       1.00      1.00      1.00         1
           C       1.00      1.00      1.00         1
           D       0.00      0.00      0.00         1
           E       1.00      1.00      1.00         1
           F       0.50      1.00      0.67         1
           G       1.00      1.00      1.00         1
           H       1.00      1.00      1.00         1
           I       1.00      1.00      1.00         1
           J       1.00      1.00      1.00         1
           K       1.00      1.00      1.00         1
           L       1.00      1.00      1.00         1
           M       1.00      1.00      1.00         1
           N       1.00      0.50      0.67         2
           O       1.00      1.00      1.00         1
           P   

D:\final project\box\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\final project\box\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\final project\box\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\final project\box\lib\site-packages\sklearn\metrics\_classification.py:1

In [15]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import os

# Load the pre-trained model
model = load_model('sign_language_main_1.h5')

# Define the class mapping dictionary
class_mapping = {
    0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H',
    8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O',
    15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V',
    22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'del', 27: 'nothing',
    28: 'space', 29: 'open_palm'
}

# Load and preprocess the image
img_path = r'Sign Language Prediction - Manasa\Test images\A519.jpg'  # Use raw string for Windows paths

# Check if the image path is valid
if not os.path.exists(img_path):
    print("Error: The image path does not exist.")
else:
    # Read the image
    img = cv2.imread(img_path)

    # Check if the image was loaded correctly
    if img is None:
        print("Error: Unable to load the image.")
    else:
        img = cv2.resize(img, (224, 224))  # Resize to match model input size
        img = np.expand_dims(img, axis=0) / 255.0  # Normalize and add batch dimension

        # Define the true label based on the filename
        filename = os.path.basename(img_path)  # Get the filename from the path
        true_label_letter = filename[0].upper()  # Extract the first character from the filename
        
        # Check if the true label letter is valid
        if true_label_letter in class_mapping.values():
            true_label = true_label_letter
        else:
            true_label = None

        # Make a prediction
        pred = model.predict(img)
        pred_class = np.argmax(pred)
        predicted_label = class_mapping[pred_class]

        # Calculate accuracy
        accuracy = 100.0 if predicted_label == true_label else 0.0

        # Display the predicted sign language letter and accuracy
        print(f'Predicted sign language letter: {predicted_label}')
        print(f'True sign language letter: {true_label}')  # Should now reflect the actual letter
        print(f'Accuracy: {accuracy}%')


1/1 [==============================] - 1s 849ms/step
Predicted sign language letter: A
True sign language letter: A
Accuracy: 100.0%


In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import os

# Load the pre-trained model
model = load_model('sign_language_main_1.h5')

# Define the class mapping dictionary
class_mapping = {
    0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H',
    8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O',
    15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V',
    22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'del', 27: 'nothing',
    28: 'space', 29: 'open_palm'
}

# Path to the folder containing test images
test_folder_path = r'extracted_archive_11\asl_alphabet_test\asl_alphabet_test'  # Update this to your actual folder path

# Specify the subset of images to test (e.g., just a few letters)
images_to_test = [
    'A_test.jpg',
    'B_test.jpg',
    'C_test.jpg',
    'H_test.jpg',
    'K_test.jpg',
    'X_test.jpg',
    'S_test.jpg',
    'O_test.jpg',
    'M_test.jpg',
    'N_test.jpg',
    'T_test.jpg',
    'V_test.jpg',
    'E_test.jpg',
    'J_test.jpg',
    'I_test.jpg',
    'U_test.jpg',
    'Z_test.jpg'
    
    # Add more image filenames as needed
]

# Variables to calculate overall accuracy
total_images = 0
correct_predictions = 0

# Iterate over the specified images
for img_name in images_to_test:
    img_path = os.path.join(test_folder_path, img_name)

    # Read the image
    img = cv2.imread(img_path)

    # Check if the image was loaded correctly
    if img is None:
        print(f"Error: Unable to load the image {img_name}.")
        continue

    img = cv2.resize(img, (224, 224))  # Resize to match model input size
    img = np.expand_dims(img, axis=0) / 255.0  # Normalize and add batch dimension

    # Define the true label based on the filename
    true_label_letter = img_name.split('_')[0].upper()  # Extract the first character before the underscore
    true_label = class_mapping.get(true_label_letter, None)

    # Make a prediction
    pred = model.predict(img)
    pred_class = np.argmax(pred)
    predicted_label = class_mapping[pred_class]

    # Count the total images and correct predictions
    total_images += 1
    if predicted_label == true_label_letter:
        correct_predictions += 1

    # Calculate accuracy for the current image
    accuracy = 100.0 if predicted_label == true_label_letter else 0.0

    # Display the predicted sign language letter and accuracy
    print(f'Image: {img_name}')
    print(f'Predicted sign language letter: {predicted_label}')
    print(f'True sign language letter: {true_label_letter}')  # Should now reflect the actual letter
    print(f'Accuracy: {accuracy}%')
    print('-' * 30)

# Calculate final accuracy and error rate
if total_images > 0:
    final_accuracy = (correct_predictions / total_images) * 100
    error_rate = 100 - final_accuracy  # Calculate error rate
    print(f'Total Images Tested: {total_images}')
    print(f'Correct Predictions: {correct_predictions}')
    print(f'Final Combined Accuracy: {final_accuracy:.2f}%')
    print(f'Error Rate: {error_rate:.2f}%')
else:
    print("No images processed.")


1/1 [==============================] - 1s 663ms/step
Image: A_test.jpg
Predicted sign language letter: A
True sign language letter: A
Accuracy: 100.0%
------------------------------
1/1 [==============================] - 0s 79ms/step
Image: B_test.jpg
Predicted sign language letter: B
True sign language letter: B
Accuracy: 100.0%
------------------------------
1/1 [==============================] - 0s 78ms/step
Image: C_test.jpg
Predicted sign language letter: C
True sign language letter: C
Accuracy: 100.0%
------------------------------
1/1 [==============================] - 0s 79ms/step
Image: H_test.jpg
Predicted sign language letter: H
True sign language letter: H
Accuracy: 100.0%
------------------------------
1/1 [==============================] - 0s 78ms/step
Image: K_test.jpg
Predicted sign language letter: K
True sign language letter: K
Accuracy: 100.0%
------------------------------
1/1 [==============================] - 0s 78ms/step
Image: X_test.jpg
Predicted sign language 